In [1]:
import common
import importlib
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import socket

# Show all columns and rows in a dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Analytics

In [2]:
# graphalytics inserts
sortledton_block_sizeonly = "num_threads_read == 0 and (hostname == 'scyper21' or hostname == 'scyper22') and library == 'sortledton' and graph in ('graph500-22', 'graph500-26', 'uniform-24')"


data = common.import_gfe("view_graphalytics_inserts").query(sortledton_block_sizeonly).copy() # data from the experiments
data["build_frequency"].fillna(pd.Timedelta(0), inplace=True) # replace NaT with 0, otherwise the records are ignored in the group by

data = data.groupby(["library", "compiler_family", "graph", "build_frequency", "block_size", "num_threads_read", "num_threads_write", "algorithm"]) \
    .agg(completion_time=("median_secs", "median"), count=("median_secs", "count"))
data = data.unstack("algorithm")[("completion_time")]
data.index.set_names("compiler", level=1, inplace=True)
data

algorithm                                                                                           bfs  \
library    compiler graph       build_frequency block_size num_threads_read num_threads_write             
sortledton gcc      graph500-22 0 days          8          0                56                 0.110322   
                                                16         0                56                 0.107629   
                                                32         0                56                 0.107772   
                                                64         0                56                 0.108521   
                                                128        0                56                 0.107036   
                                                256        0                56                 0.107557   
                                                512        0                56                 0.104051   
                                                1024       0                56                 0.105758   
                    graph500-26 0 days          64         0                56                 1.546463   
                                                128        0                56                 1.566064   
                                                256        0                56                 1.529622   
                                                512        0                56                 1.514151   
                                                1024       0                56                 1.519648   
                    uniform-24  0 days          256        0                56                 0.411999   
                                                512        0                56                 0.408690   
                                                1024       0                56                 0.406473   

algorithm                                                                                            cdlp  \
library    compiler graph       build_frequency block_size num_threads_read num_threads_write               
sortledton gcc      graph500-22 0 days          8          0                56                   6.091867   
                                                16         0                56                   8.485812   
                                                32         0                56                   4.681293   
                                                64         0                56                   4.840380   
                                                128        0                56                   4.722669   
                                                256        0                56                   4.991100   
                                                512        0                56                   4.834921   
                                                1024       0                56                   7.365357   
                    graph500-26 0 days          64         0                56                 110.131614   
                                                128        0                56                 150.444945   
                                                256        0                56                 108.600880   
                                                512        0                56                 153.776543   
                                                1024       0                56                 152.119144   
                    uniform-24  0 days          256        0                56                  23.897964   
                                                512        0                56                  45.057128   
                                                1024       0                56                  36.975817   

algorithm                                                                                              lcc  \


#### Observation

There is no trend that indicates block sizes between 256 and 1024 have major influence on the performance of analytical algorithms.

Below 256 multiple analytical algoritms loose performance. 

## Inserts

In [4]:
sortledton_block_sizeonly = "(hostname == 'scyper21' or hostname == 'scyper22') and library == 'sortledton' and graph in ('graph500-22', 'graph500-26', 'uniform-24')"

edges_per_graph = pd.DataFrame({
    "graph": ["friendster", "dota-league", "graph500-22", 
              "graph500-24", "graph500-26", "uniform-22", 
              "uniform-24", "uniform-26"
              ],
    "edges": [1806067135, 50870313, 64155735, 260379520, 1051922853, 64155735, 260379520, 1051922853]
})

data = common.import_gfe("View_Inserts").query(sortledton_block_sizeonly).copy() # data from the experiments
data["build_frequency"].fillna(pd.Timedelta(0), inplace=True) # replace NaT with 0, otherwise the records are ignored in the group by
data = data.merge(edges_per_graph, on="graph")
data["edges_per_second"] = data["edges"] / data["completion_time_secs"]

data_grouped = data.groupby(["compiler_family", "graph", "library", "block_size", "num_threads"]) \
    .agg(edges_per_second=("edges_per_second", "median"), count=("edges_per_second", "count"))
#data = data.unstack("algorithm")[("completion_time")]
data_grouped.index.set_names("compiler", level=1, inplace=True)
data_grouped

edges_per_second  \
compiler_family compiler    library    block_size num_threads                     
gcc             graph500-22 sortledton -1         56               3.485842e+06   
                                        8         56               7.834754e+05   
                                        16        56               1.745052e+06   
                                        32        56               3.449060e+06   
                                        64        56               4.768647e+06   
                                        128       56               4.987679e+06   
                                        256       56               4.509551e+06   
                                        512       56               4.232109e+06   
                                        1024      56               3.570821e+06   
                graph500-26 sortledton -1         56               3.028517e+06   
                                        64        56               3.990773e+06   
                                        128       56               4.421168e+06   
                                        256       56               4.159755e+06   
                                        512       56               4.003019e+06   
                                        1024      56               3.015199e+06   
                uniform-24  sortledton  256       56               5.831318e+06   
                                        512       56               5.712393e+06   
                                        1024      56               5.641006e+06   

                                                               count  
compiler_family compiler    library    block_size num_threads         
gcc             graph500-22 sortledton -1         56              14  
                                        8         56               1  
                                        16        56               1  
                                        32        56               1  
                                        64        56               1  
                                        128       56               1  
                                        256       56               1  
                                        512       56               1  
                                        1024      56               2  
                graph500-26 sortledton -1         56               4  
                                        64        56               1  
                                        128       56               1  
                                        256       56               1  
                                        512       56               1  
                                        1024      56               2  
                uniform-24  sortledton  256       56               1  
                                        512       56               1  
                                        1024      56               2

#### Observation

For uniform graphs the block size has no influence on the insertions performance. 
For power-law graphs we see that blocks of sizes of 128 or 256 are best.

#### Explanation

For uniform graphs we do not see the use of many/any blocks because the adjacency set sizes are mostly below.
For power-law graphs small block sizes are better because either finding the correct block is cheaper if performed with skip-list jumps than binary search or/xor there is less data to move, e.g. in the GC.
For too small block sizes we see more random jumps to check the skip list and more memory allocation overhead.

## Result

We run a wider range of block size parameters to
  (1) find the limit when it influences analytical performance.
  (2) find the best block size for insertions

There is no need to run with blocks bigger than 1024 because we do expect insertion performance to decrease without winning analytical performance.
  
We start with the parameters: 8, 32, 64 and 128 on the same graphs.

We find that 512 is the best block size because it is safe to state that analytical algorithms do not suffer and we see good insertion performance.

## Open Question

What happens to the storage size of the data structure?
 We already observed that a block size of 32 cannot be loaded into memory for graph500-26.
What about LCC?